This ipython notebook is used for iQNet tutorial. Please email Bin Liu (bliu8@ncsu.edu) if you have any question about this script. 

#Dependencies and Pre-requisites
The iQNet is built on Tensorflow 2.0

In [ ]:
import numpy as np
from sklearn.externals.joblib import dump, load

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow.compat.v2 as tf
import tensorflow.keras as tfk
tf.enable_v2_behavior()
from keras.layers import Activation, Dense, Input
from keras.models import Sequential, Model
from keras import backend as K
from keras.models import load_model
from keras.utils import plot_model
from scipy.signal import medfilt

#Data Preprocessing
The iQNet is used to predict quasar continua within the rest-frame wavelength range [1180, 1600] Angstroms with spatial resolution of 0.05 Angstrom/pixel. Thus, please make sure you transfer the input spectra into rest frame and then bin them appropriately. The input has to be 2D Numpy array, where the rows represent the number of spectra and the columns are the number of pixels per spectrum.

In [ ]:
wave = np.arange(1180, 1600, 0.05)
wave.shape

(11600,)

The iQNet only takes redward spectra of Lya peak so make sure to feed the correct input spectra. You can also custom this for your own project, however you need to use redward spectra of Lya emission as model input.

In [ ]:
# index to split red/blue sides
split_idx = np.where(wave<1216)[0][-1]

In [ ]:
your_red_spectra = your_spectra[:, :split_idx]

Note: Please normalize your input spectra around 1280 Angstrom at rest-frame wavelength as this is one of the pre-processing requirement for iQNet-V1 because it is trained with all training spectra normalized at ~1280 Angstrom in rest frame.

#iQNet Prediction

Initialize the iQNet model

In [ ]:
from model.iqnet_setup import iqnet_model, load_iqnet_weights

In [ ]:
iqnet_main, iqnet_encoder, iqnet_decoder = iqnet_model(your_red_spectra.shape[-1], # input dimension
                                                       your_spectra.shape[-1]) # output dimension

Now it's time to load the iQNet weights. There are two available trained models, one trained with HST Cosmic Origin Spectrograph QSO spectra, and the other one trained with HST + SDSS QSO spectra. You can try either model and just specify the keyword name when you initialize and load the model weigths. The keyword 'hst' is reserved for iQNet-V1 trained with HST spectra, and 'hst+sdss' is for iQNet-V1 trained with HST and SDSS spectra.



In [ ]:
load_iqnet_weights(iqnet_main, name='hst+sdss')

If no error exists, please proceed to use iQNet for continuum prediction as follows:

In [ ]:
iqnet_out = iqnet_main.predict(your_input_to_iqnet)

# Median filter may not be used on your decision
iqnet_pred_mf = np.array([medfilt(i, 49) for i in iqnet_pred])

You may notice the predicted continua are different from those two provided models. This is because they are trained with different training spectra. While model structures affect the predictions to some extent, the key factor that influence the predicted continua are the training spectra. For better accuracy and precision, you may need to retrain the model further to adapt to your own dataset. This will reduce the effect of out-of-distribution predictions.

The idea of iQNet-V1 is to help scientists quickly set up a deep learning neural network (DNN) model without going into details. Our easy-to-go setup pipeline helps people who do not have deep learning background to build a DNN model quickly.